## essentials

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import networkx as nx


from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.base import clone
from sklearn.metrics import roc_auc_score

import statsmodels.api as sm
from statsmodels.stats.multitest import fdrcorrection
from statsmodels.formula.api import ols

import sys
sys.path.append("..")
sys.path.append("../..")
from cluster_analysis import *
# from LOR_calculation import *

import warnings

2024-05-02 08:03:58.841055: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-02 08:03:58.842514: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-02 08:03:58.883262: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-02 08:03:59.055161: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-02 08:03:59.756628: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

trimethoprim

In [18]:
species='Escherichia_coli'

X_df = pd.read_csv(f'../../data/temp/X_{species}_SxG_filtered_hypo_unknowns_freq5.csv', index_col=0) # filtering

drug='trimethoprim'

pheno_df= pd.read_csv(f'../../metadata/{species}/{species}_{drug}.csv', index_col=0)
y_df=pheno_df
y_df.index = y_df.index.astype('float')

y_df = y_df.sort_index()

y_indices=list(y_df.index)


X_df = X_df.sort_index()
y_df = y_df.sort_index()

y_indices=list(y_df.index)
X_indices=list(X_df.index)

intersection = [i for i in y_indices if i in X_indices]
y_df = y_df.loc[intersection]
X_df = X_df.loc[intersection]

X_df = X_df.sort_index()
y_df = y_df.sort_index() # -- just making sure bcs im paranoid

X = X_df.values
y = y_df.values



labeled_matrix = pd.concat([X_df, y_df], axis=1)
labeled_matrix.shape

(499, 18876)

In [4]:
clstr_df= get_cluster_representatives(f'../../pangenome-repo/Pangenome-Analysis-Workflow/codes/{species}/{species}.fasta.clstr')
_product_df=get_representative_products(f'../../pangenome-repo/Pangenome-Analysis-Workflow/codes/{species}/{species}.fasta')
product_df = combine_cluster_product(clstr_df, _product_df)
freq_df = get_cluster_frequency(f'../../pangenome-repo/Pangenome-Analysis-Workflow/codes/{species}/{species}_cluster_frequencies.csv')

#  --counting those thar have hypotehtical in the product name
product_df['hypothetical'] = product_df['product_name'].str.contains('hypothetical', case=False)
product_df['hypothetical'].sum()

#  --same for unknown
product_df['unknown'] = product_df['product_name'].str.contains('unknown', case=False)
product_df['unknown'].sum()

# now will get a list of clusters that are either hypo or unknown to filter out
hypothetical_clusters = product_df[product_df['hypothetical']==True].index.tolist()
unknown_clusters = product_df[product_df['unknown']==True].index.tolist()

to_remove = list(set(hypothetical_clusters).union(unknown_clusters))

# -- get the list of clusters that are present in less than 5 genomes
low_freq_clusters = freq_df[freq_df['frequency']<5].index.tolist()

to_remove = list(set(to_remove).union(low_freq_clusters))

product_df = product_df.drop(to_remove)

In [5]:


def transform_cluster_to_product(cluster_name, clstr_product_df):
    #match it ffrom clstr_poduct_df from the index
    cluster = clstr_product_df.loc[cluster_name]
    product=cluster['product_name']
    
    return product

## MI

In [23]:
# -- computing MI between each col and the last col in labeled_matrix
from sklearn.metrics import mutual_info_score

mi_scores = {}
for col in labeled_matrix.columns[:-1]:
    mi_scores[col]=mutual_info_score(labeled_matrix[col], labeled_matrix['SIR'])


MI_top_100 = sorted(mi_scores, key=mi_scores.get, reverse=True)[:100]

for i in  range(len(MI_top_100)):
    MI_top_100[i]=transform_cluster_to_product(MI_top_100[i], product_df)

In [25]:
MI_top_100

['Dihydropteroate synthase type-2 (EC 2.5.1.15) @ Sulfonamide resistance protein_7',
 'Integron integrase IntI1_2',
 'Aminoglycoside 6-phosphotransferase (EC 2.7.1.72) => APH(6)-Ic/APH(6)-Id_11',
 'Small multidrug resistance (SMR) efflux transporter => QacE delta 1, quaternary ammonium compounds_4',
 'Dihydropteroate synthase type-2 (EC 2.5.1.15) @ Sulfonamide resistance protein_4',
 'Mercuric transport protein, MerC_1',
 "Aminoglycoside 3''-nucleotidyltransferase (EC 2.7.7.-) => ANT(3'')-Ia (AadA family)_6",
 'Chromate transport protein ChrA_3',
 "Aminoglycoside 3''-phosphotransferase (EC 2.7.1.87) => APH(3'')-I_1",
 'Mercuric transport protein, MerT_2',
 'Periplasmic mercury(+2) binding protein, MerP_1',
 'Mercuric ion reductase (EC 1.16.1.1)_1',
 'Mercuric resistence transcriptional repressor, MerD_1',
 'Mercuric resistance operon regulatory protein MerR_1',
 'T6SS component TssM (IcmF/VasK)_18',
 "Macrolide 2'-phosphotransferase => Mph(A) family_1",
 'Uncharacterized GST-like prote

In [32]:
arg = ['Multidrug efflux system, inner membrane proton/drug antiporter (RND type) => MexB of MexAB-OprM',
 'Dihydrofolate reductase (EC 1.5.1.3)',
 'trimethoprim-resistant dihydrofolate reductase DfrA1']

def parse_ARGs(ARG_products:pd.DataFrame, top_genes:pd.DataFrame):
    '''
    Takes in a list of ARG products and a list of top genes and returns a dict of ARGs that are in the top genes (with their rank)
    '''
    ARGs = {}
    # print(ARG_products)
    for gene in top_genes:
        for product in ARG_products:
            # print(len(product))
            if product in gene:
                ARGs[top_genes.index(gene)]=gene
    return ARGs

parse_ARGs(arg, MI_top_100)

{20: 'Dihydrofolate reductase (EC 1.5.1.3)_1'}

# test

In [33]:
presence_path="../../data/temp/X_Escherichia_coli_SxG_filtered_hypo_unknowns_freq5.csv" #add the method that generates presence to this script
presence_df= pd.read_csv(presence_path, index_col=0)
presence_df = presence_df.T

presence_df.to_csv("../../data/temp/X_Escherichia_coli_GxS_filtered_hypo_unknowns_freq5.csv")

In [35]:
species = "Escherichia_coli"; drug="trimethoprim"
  
gene_pairs=[]
with open(f'../../data/temp/SVM_{species}_{drug}_top_1000_gene_pairs.txt', 'r') as f:
    for line in f:
        gene_pairs.append(tuple(line.strip().split("-")))

def extract_genes_from_pairs(gene_pairs:list)->list:
    '''
    takes a list of gene pairs and returns a list of all genes in the gene pairs

    param:
    ------
        - gene_pairs: list of tuples

    return:
    ------
        - genes: list
    '''
    genes=[]
    for pair in gene_pairs:
        gene1=pair[0]
        gene2=pair[1]
        genes.append(gene1)
        genes.append(gene2)

    genes= list(set(genes))
    return genes

genes = extract_genes_from_pairs(gene_pairs)

In [37]:
len(genes)

1000

In [42]:
with open(f'../../data/temp/SVM_{species}_{drug}_top_1000_genes.txt', 'r') as f:
    gene_list = f.readlines()
    genes = [line.strip() for line in gene_list]